In [6]:
# Step 1: Import necessary libraries
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
import hdbscan
import numpy as np
import time
from sklearn.preprocessing import StandardScaler


In [7]:
df=pd.read_excel('nadilFInalizedDataset.xlsx')

In [8]:
df.head()

,date,desiption,debit,credit,balance
0,2022-09-29,CSH DEP,NaN,1000.00,1000
1,2022-10-31,TRF,NaN,17480.00,18480
2,2022-10-31,INT,NaN,4.64,18484.64
3,2022-11-06,t ahirt OTHBNK T,6030.0,NaN,12454.64
4,2022-11-06,010001088282101 OTHBNK T,3030.0,NaN,9424.64


In [9]:
# Step 3: Data Preprocessing - Normalize Text & Expand Abbreviations

# Define abbreviation dictionary
abbreviations = {
    'PYT': 'Payment',
    'TRF': 'Transfer',
    'DEP': 'Deposit',
    'WDL': 'Withdrawal',
    'WD': 'Withdrawal',
    'POS': 'Point of Sale',
    'ATM': 'ATM Withdrawal',
    'CHQ': 'Cheque',
    'DD': 'Demand Draft',
    'BT': 'Bank Transfer',
    'ACH': 'Automated Clearing House',
    'NEFT': 'National Electronic Funds Transfer',
    'RTGS': 'Real-Time Gross Settlement',
    'IMPS': 'Immediate Payment Service',
    'UPI': 'Unified Payments Interface',
    'INT': 'Interest',
    'CHG': 'Charge',
    'FEE': 'Fee',
    'TXN': 'Transaction',
    'REV': 'Reversal',
    'EMI': 'Equated Monthly Installment',
    'CC': 'Credit Card',
    'POS REF': 'Point of Sale Refund',
    'BIL': 'Bill Payment',
    'BILP': 'Bill Payment',
    'INV': 'Investment',
    'REF': 'Refund',
    'SAL': 'Salary Credit',
    'SL': 'Salary Credit',
    'TFR': 'Transfer'
}

# Function to clean text
def clean_text(text, abbr_dict):
    text = str(text).lower()  # Convert to lowercase
    for abbr, full_form in abbr_dict.items():
        text = re.sub(rf'\b{abbr.lower()}\b', full_form.lower(), text)
    return text

# Apply text cleaning to the 'Particulars' column
df['cleaned_particulars'] = df['desiption'].apply(lambda x: clean_text(x, abbreviations))


In [11]:
# Step 4: Separate Payments and Receipts Transactions

# Payments: Non-null 'Payments' column, null 'Receipts'
df_debit = df[df['debit'].notna() & df['credit'].isna()].copy()

# Receipts: Non-null 'Receipts' column, null 'Payments'
df_credit = df[df['credit'].notna() & df['debit'].isna()].copy()


In [12]:
df_credit.head()

,date,desiption,debit,credit,balance,cleaned_particulars
0,2022-09-29,CSH DEP,NaN,1000.00,1000,csh deposit
1,2022-10-31,TRF,NaN,17480.00,18480,transfer
2,2022-10-31,INT,NaN,4.64,18484.64,interest
6,2022-11-16,TRF,NaN,2587.50,11987.14,transfer
8,2022-11-30,INT,NaN,20240.00,31777.14,interest


In [13]:
df_debit.head()

,date,desiption,debit,credit,balance,cleaned_particulars
3,2022-11-06,t ahirt OTHBNK T,6030.0,NaN,12454.64,t ahirt othbnk t
4,2022-11-06,010001088282101 OTHBNK T,3030.0,NaN,9424.64,010001088282101 othbnk t
5,2022-11-15,RIB/RMB SE.CH 20 IBMB Chg,25.0,NaN,9399.64,rib/rmb se.ch 20 ibmb charge
7,2022-11-18,nadil Siriwardha MB SA TF,450.0,NaN,11537.14,nadil siriwardha mb sa tf
11,2022-12-24,nadil OTHBNK T,7530.0,NaN,27264.9,nadil othbnk t


In [15]:
# Step 5: Generate Sentence Embeddings

# Initialize Sentence Transformer model
model = SentenceTransformer('gtr-t5-large')

# Generate embeddings for Payments
credit_embeddings = model.encode(df_credit['cleaned_particulars'].tolist())

# Generate embeddings for Receipts
debit_embeddings = model.encode(df_debit['cleaned_particulars'].tolist())


In [17]:
# Step 6: Apply HDBSCAN Clustering for credit

# Standardize embeddings
scaler_credit = StandardScaler()
credit_embeddings_scaled = scaler_credit.fit_transform(credit_embeddings)

# Apply HDBSCAN
hdbscan_credit = hdbscan.HDBSCAN(min_cluster_size=2, gen_min_span_tree=True)
df_credit['Cluster'] = hdbscan_credit.fit_predict(credit_embeddings_scaled)


In [18]:
# Step 7: Apply HDBSCAN Clustering for debit

# Standardize embeddings
scaler_debit = StandardScaler()
debit_embeddings_scaled = scaler_debit.fit_transform(debit_embeddings)

# Apply HDBSCAN
hdbscan_debit = hdbscan.HDBSCAN(min_cluster_size=5, gen_min_span_tree=True)
df_debit['Cluster'] = hdbscan_debit.fit_predict(debit_embeddings_scaled)


In [24]:
# Step 8: Print Clusters for Payments
print("\n### Credit Clusters ###")

for cluster in sorted(df_credit['Cluster'].unique()):
    print(f"\nCluster {cluster}:")
    cluster_data = df_credit[df_credit['Cluster'] == cluster]
    print(cluster_data[['date','desiption', 'debit', 'credit','balance']])



### Credit Clusters ###

Cluster -1:
          date                  desiption  debit   credit   balance
14  2022-12-28            Harsha EFT OTHE    NaN  10000.0   58234.9
45  2023-03-02        Gajanayake EFT OTHE    NaN   6000.0  69048.99
89  2023-07-03  Boardim fees sav EFT OTHE    NaN   6000.0  39110.39
113 2023-08-05          069RB01 DEP (NOV)    NaN  12000.0  44163.23
114 2023-08-05  savindu board fe EFT OTHE    NaN   6000.0  50163.23
134 2023-08-19              susi EFT OTHE    NaN   1000.0  12803.23
146 2023-08-25           ROSHANA EFT OTHE    NaN    500.0  11993.23
158 2023-09-02            router EFT OTHE    NaN  11000.0  13297.97
159 2023-09-02  From Loku appach EFT OTHE    NaN   5000.0  18297.97
178 2023-09-22        water bill EFT OTHE    NaN    500.0   5590.47
184 2023-09-30         0022RB01 DEP (NOV)    NaN   5600.0  11870.47
230 2023-10-31                        INI    NaN     39.0   1039.35
232 2023-11-01             Apple EFT OTHE    NaN   1500.0    2537.4
273 2023-1

In [25]:
# Step 9: Print Clusters for Receipts
print("\n### Debit Clusters ###")

for cluster in sorted(df_debit['Cluster'].unique()):
    print(f"\nCluster {cluster}:")
    cluster_data = df_debit[df_debit['Cluster'] == cluster]
    print(cluster_data[['date','desiption', 'debit', 'credit','balance']])



### Debit Clusters ###

Cluster -1:
          date                  desiption     debit  credit   balance
7   2022-11-18  nadil Siriwardha MB SA TF    450.00     NaN  11537.14
15  2022-12-28                    CSH WDR  10000.00     NaN   48234.9
19  2022-12-29                    CSH WDR   5000.00     NaN  51891.15
22  2023-01-06              film OTHBNK T   3130.00     NaN   65435.2
23  2023-01-08              film OTHBNK T   3030.00     NaN   62405.2
27  2023-01-31                    CSH WDR  21200.00     NaN   73015.2
31  2023-02-15                    CSH WDR   3850.00     NaN  72691.63
35  2023-02-28                    C5H WDR    600.00     NaN  60001.63
48  2023-03-07   nadil , roshana OTHBNK T  12030.00     NaN  44883.99
98  2023-07-22    04199071001696 POS Trns   6500.00     NaN  36760.39
108 2023-08-01        0713201322 BILL PYT    200.00     NaN  50723.23
138 2023-08-20        0723728777 BILL PYT    200.00     NaN  16343.23
153 2023-08-31    004199071001696 ATM WDR   3030.00  